<img src="../../shared/img/banner.svg"></img>

# Homework 03 - Models IRL

In [ ]:
%matplotlib inline

In [ ]:
import sys

sys.path.append("../../")

from shared.src import quiet
from shared.src import seed

In [ ]:
import random

from client.api.notebook import Notebook
from IPython.display import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import seaborn as sns

import shared.src.utils.util as util

In [ ]:
ok = Notebook("ok/config")

## Learning Objectives


1. Practice building models of random processes in `pyMC` based on descriptions of the model.
1. Learn to recognize problems in real life that you can use models to solve.

## Section 1 - Warm-Up

### What's the most you ever lost in a coin toss?

To begin, write a model for what might be considered the simplest random process:
the tossing of a fair coin.

Remember that `pm.Categorical` is used to make random variables that take on one of a limited set of values. Look back at the lecture notes if you need a reminder of how to make a model of coin tossing.

```python
coin_toss_model = pm.Model()

with coin_toss_model:
    coin = pm.Categorical(name="coin", ?)
```

In [ ]:
coin_toss_model = pm.Model()

with coin_toss_model:
    pass

In [ ]:
ok.grade("q1")

Now, draw some samples from this model and plot the numbers of heads and tails.
Use `util.samples_to_dataframe` to convert samples into a dataframe.

### 1 + 1 = ?

Random phenomena are always intertwined with deterministic phenomena:
when we throw a dart at a dartboard, the place it lands is random, but the number of points you get for hitting a particular point is not random.
As we write models for random phenomena, therefore, we'll need to often apply
deterministic transformations.

Thanks to the flexibility and generality of `pymc3` we can, with sufficient care,
even make models that are totally deterministic!

In the cell below, write a model called `adding_model` that adds together two `Categorical` random variables, `X` and `Y`, that are always equal to `1` to create a random variable `Z` that is always equal to `2`. Remember to use `pm.Deterministic`!

The result should look something like the template below.

```python
adding_model = pm.Model()

with adding_model:
    X = pm.Categorical(name="X", ?)
    Y = pm.Categorical(name="Y", ?)
    Z = ?
```

In [ ]:
ok.grade("q2")

Use the cells below to draw two samples from the model, then print the results.

The winner of a game is often the person with the most points at the end.
So a deterministic transformation we'll need if we want to model the winner's score is "take the largest number", or `maximum`.
Of course, there are also games like golf, so we should also be able to take the smallest number, aka `minimum`.

In the cell below, use `pm.math.maximum` in a model called `max_model` to select the larger of two `Categorical` random variables, `X` and `Y`, where `X` always equal to `0` and where `Y` is always equal to `1`. Use the template below if you run into trouble.

```python
max_model = pm.Model()

with max_model:
    X = pm.Categorical(name="X", ?)
    Y = pm.Categorical(name="Y", ?)
    Z = pm.Deterministic(name="Z", var=pm.math.maximum(?, ?))
```

Use the cells below to draw two samples from the model, then print the results.

In [ ]:
ok.grade("q3")

## Section 2 - Root: A Game of Woodland Might and Right

In [ ]:
Image("./img/rootlogo.png", width=500)

[*Root*](https://ledergames.com/root/) is a board game that combines the aesthetics of *Redwall* or *Watership Down* with the mechanics of *Risk*.
Players control armies of adorable woodland creatures vying for political supremacy over the forest.

When two players do battle, two die are rolled, numbered 0 through 3.
One player is the attacker and the other is the defender.
The attacker deals a number of "hits" to the defending army equal to the number on the die that rolled higher,
while the defender deals a number of "hits" to the attacking army equal to the number on the die that rolled lower.

As the wise gener-owl of the Bird Army, you'd like to apply your modeling skills to predict the outcomes of battles and guide your strategic planning.

In the cell below, write a model for a *Root* battle.
The die rolls should be `Categorical` or `DiscreteUniform`, while the number of hits for the attacker and defender should be a `Deterministic` transform of the die rolls (`pm.math.minimum` and `.maximum` will come in handy here).
Name the variable representing the number of attacker hits `"attacker"` and the variable representing the number of defender hits `"defender"`, or the autograder tests will fail.

In [ ]:
root_model = pm.Model()

with root_model:
    pass

In [ ]:
ok.grade("q4")

Now, sample from your model and convert the samples into a dataframe with `util.samples_to_dataframe`.
Take at least 1000 samples.

Then, compute the average number of hits for the attacker and the defender (as `attacker_mean` and `defender_mean`) and the chance that the attacker deals no hits (as `attacker_chance_zero`).

In *Root*, one of the goals is to have more soldiers left over at the end of a battle than your enemy does.
Each hit removes one soldier.
So one measurement that is of particular interest for your strategic planning is the difference in the number of hits you score and the number of hits your opponent scores.

In the cell below,
compute the average and the median difference in the number of hits.

In [ ]:
ok.grade("q5")

## Section 3 - Caffeine and Controversy

Like many scientists, of both the data and research variety,
you have a close relationship with caffeine.
It helps you stay alert and focused while you're working.

But unlike other scientists, you also study caffeine!
Since you're a psychologist, you study its effects on the mind,
in particular on a measure of alertness based on a task.

The first, simple model of how caffeine changes alertness is that,
on average, alertness increases.
In this model, nothing else changes about
the distribution of alertness before and after caffeine:
there's no change in the spread, no increase in the chance of very high or low values,
no change in the "shape" of the distribution.

In such a simple model, it's typical to assume that the shape is _Gaussian_, or normal.

The cell below defines such a simple model of the effect of caffeine on alertness.

The average alertness score, with and without caffeine, is defined by the `Deterministic` variable `mus`.
Notice that, for technical reasons, the Python _list_ that stores those values
needs to be passed to `to_pymc` during the definition of the `pyMC` _variable_ that stores those values.
This will be important in the next question.

Then, for each observation, we "flip a coin", called `has_caffeine`, to determine whether
the observation corresponds to an individual who had caffeine or not.

Lastly, we determine the `alertness_score`.
In our model, the only thing about the distribution of alertness that can change
depending on `has_caffeine` is the mean, `mu`,
and we implement this in `pyMC` by using the same variable, no matter what the value of `has_caffeine` is,
and by using the same value for `sd`, again no matter what the value of `has_caffeine` is.

In [ ]:
first_caffeine_model = pm.Model()

mu_values = [10, 12]

with first_caffeine_model:
    mus = pm.Deterministic(name="mus", var=util.to_pymc(mu_values))
    has_caffeine = pm.Categorical(name="has_caffeine", p=[1/2, 1/2])
    alertness_score = pm.Normal("alertness_score", mu=mus[has_caffeine], sd=0.5)

Take samples from this model and visualize the results.
Make sure you split up the data according to the value of `has_caffeine`,
e.g.

```python
sns.stripplot(first_caffeine_samples.alertness_score, first_caffeine_samples.has_caffeine, orient="h")
```

Imagine a controversy in the world of caffeine research:
one scientist claims that, contrary to the simple model used above,
caffeine changes more than just the average alertness.

Instead, they say, it both increases average alertness _and_ the variability in alertness,
resulting in some individuals whose alertness acutally decreases when given caffeine.
This is perhaps, they speculate, because they become too jittery to focus correctly.

In the cells below, define a `second_caffeine_model` that generates data according to the model
of this contrarian scientist, draw samples, and visualize the data as you did for the `first_caffeine_model`.

The template below should get you started.
Use the same `mu_values` as before,
but set the variable `sds` using the `sd_values` in the template.
When setting the value of `sd` for `alertness_score`, use the same style as used to set the value of `mu`.

```python

second_caffeine_model = pm.Model()
sd_values = [0.5, 2]

with second_caffeine_model:
    mus = pm.Deterministic(name="mus", var=util.to_pymc(mu_values))
    sds = pm.Deterministic(name="sds", var=?)
    has_caffeine = pm.Categorical(name="has_caffeine", p=[1/2, 1/2])
    alertness_score = pm.Normal("alertness_score", mu=mus[has_caffeine], sd=?)
```

In [ ]:
ok.grade("q6")

Note: there is a small but non-negligible chance that the sampler will fail, under the hood, even if the model is written correctly. If, during grading, you receive an error message that says you should `increase 'target_accept' or reparameterize` AND, at the same time, do not get credit for the problem, run the grading code again a few times. The message may remain, but you should get credit every other time.

If this happens to you more than once, alert course staff so they can check whether the issue is with your code or with the grading code.

In [ ]:
ok.score()